<a href="https://colab.research.google.com/github/abhilashhn1993/depression-assessment-on-twitter/blob/Dragon/Code/Final_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re
import string
import datetime

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.svm import SVC

from sklearn import model_selection, naive_bayes, svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [3]:
def performCV(X, Y, model, folds, measure):
  return cross_val_score(model, X, Y, cv=folds, scoring=measure)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [179]:
import io
df = pd.read_excel(io.BytesIO(uploaded['Pre_Post_CG_Classifier.xlsx']), sheet_name='post-cg_AgeGender')

In [ ]:
df.head()

In [ ]:
df.shape

In [182]:
#Data pre-processing
#Leave out Twitter_handle column from the classifier 
df = df.drop(['Twitter_handle'], axis=1)

**###############OPTIONAL(SKIP IF NOT NEEDED)########################**

**Performing One hot encoding to convert categorical features for modeling**

In [183]:
new_cols = ['Age_group','Gender']

In [ ]:
df_proc = pd.get_dummies(df, prefix_sep="_", columns=new_cols)
df_proc.head()

In [185]:
features_for_modeling = list(range(0, df_proc.shape[1]-1))

In [ ]:
#Seperate Train and target from the dataframe
df_proc_train = df_proc.iloc[:,features_for_modeling]
df_proc_train = df_proc.drop(['Target'], axis=1)
df_proc_train.head()

In [ ]:
df_proc_target = df_proc.loc[:,['Target']]
df_proc_target.head()

In [ ]:
df_proc_train.shape

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df_proc_train,df_proc_target,test_size=0.3)
sum(Test_Y['Target'] == 1)

In [ ]:
Train_Y.Target.value_counts()

**SKIP to classifier after this line of code**

**###############OPTIONAL CODE ENDS########################**

In [143]:
features_for_modeling = list(range(0, df.shape[1]-1))

In [ ]:
#Seperate Train and target from the dataframe
df_train = df.iloc[:,features_for_modeling]
df_train = df.drop(['Target'], axis=1)
df_train.head()

In [ ]:
df_target = df.loc[:,['Target']]
df_target.head()

In [ ]:
df_train.shape

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df_train,df_target,test_size=0.3)
sum(Test_Y['Target'] == 1)

In [ ]:
Train_Y.Target.value_counts()

**CLASSIFIERS**

**Naive Bayes Classifier**

In [220]:
from sklearn.naive_bayes import GaussianNB
model =  GaussianNB()

In [ ]:
nb = model.fit(Train_X, Train_Y)
y_pred_nb = nb.predict(Test_X)

In [ ]:
print("Accuracy:",metrics.accuracy_score(Test_Y, y_pred_nb))

In [ ]:
precision, recall, fscore, support = score(Test_Y, y_pred_nb)
print("Recall: ", recall.mean(), "Precision: ", precision.mean(), "F-score: ", fscore.mean())

In [ ]:
nb_roc_auc = roc_auc_score(Test_Y, model.fit(Train_X, Train_Y).predict(Test_X))
fpr, tpr, thresholds = roc_curve(Test_Y, model.fit(Train_X, Train_Y).predict_proba(Test_X)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='NB (area = %0.2f)' % nb_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig('NB_post-cg_Topics_AgeGender')
plt.show()

**NB with Cross Validation**

5 fold CV

In [ ]:
from sklearn.naive_bayes import GaussianNB
model =  GaussianNB()

accuracy = performCV(Train_X, Train_Y, model, 5, 'accuracy')
recall = performCV(Train_X, Train_Y, model, 5, 'recall')
precision = performCV(Train_X, Train_Y, model, 5, 'precision')
f1score = performCV(Train_X, Train_Y, model, 5, 'f1')

In [ ]:
print("Accuracy: ", accuracy.mean(), "Recall: ", recall.mean(), "Precision: ", 
      precision.mean(), "f1-score: ", f1score.mean())

10 fold CV

In [ ]:
accuracy = performCV(Train_X, Train_Y, model, 10, 'accuracy')
recall = performCV(Train_X, Train_Y, model, 10, 'recall')
precision = performCV(Train_X, Train_Y, model, 10, 'precision')
f1score = performCV(Train_X, Train_Y, model, 10, 'f1')

In [ ]:
print("Accuracy: ", accuracy.mean(), "Recall: ", recall.mean(), "Precision: ", 
      precision.mean(), "f1-score: ", f1score.mean())

**Logistic Regression**

In [ ]:
logreg = LogisticRegression()
logreg.fit(Train_X, Train_Y)

In [ ]:
y_pred = logreg.predict(Test_X)
print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(logreg.score(Test_X, Test_Y)))

In [ ]:
precision, recall, fscore, support = score(Test_Y, y_pred)
print("Recall: ", recall.mean(), "Precision: ", precision.mean(), "F-score: ", fscore.mean())

In [ ]:
logit_roc_auc = roc_auc_score(Test_Y, logreg.predict(Test_X))
fpr, tpr, thresholds = roc_curve(Test_Y, logreg.predict_proba(Test_X)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='LR (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig('LR_post-cg_Topics_AgeGender')
plt.show()

**Logistic Regression with Cross Validation**

5-fold


In [ ]:
logreg = LogisticRegression()

accuracy = performCV(Train_X, Train_Y, logreg, 5, 'accuracy')
recall = performCV(Train_X, Train_Y, logreg, 5, 'recall')
precision = performCV(Train_X, Train_Y, logreg, 5, 'precision')
f1score = performCV(Train_X, Train_Y, logreg, 5, 'f1')

In [ ]:
print("Accuracy: ", accuracy.mean(), "Recall: ", recall.mean(), "Precision: ", 
      precision.mean(), "f1-score: ", f1score.mean())

10-fold


In [ ]:
accuracy = performCV(Train_X, Train_Y, logreg, 10, 'accuracy')
recall = performCV(Train_X, Train_Y, logreg, 10, 'recall')
precision = performCV(Train_X, Train_Y, logreg, 10, 'precision')
f1score = performCV(Train_X, Train_Y, logreg, 10, 'f1')

In [ ]:
print("Accuracy: ", accuracy.mean(), "Recall: ", recall.mean(), "Precision: ", 
      precision.mean(), "f1-score: ", f1score.mean())

**Combination of Models**

In [ ]:
plt.figure()
models = [
          {
              'label': 'LR',
              'model': LogisticRegression(),
          },
          {
              'label': 'NB',
              'model': GaussianNB(),
          }
]

for m in models:
  model = m['model']
  model.fit(Train_X, Train_Y)
  y_pred = model.predict(Test_X)

  fpr, tpr, thresholds = metrics.roc_curve(Test_Y, model.predict_proba(Test_X)[:,1])

  auc = metrics.roc_auc_score(Test_Y, model.predict(Test_X))

  plt.plot(fpr, tpr, label='%s (area = %0.2f)' % (m['label'], auc))

# Custom settings for the plot
#plt.figure() 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig('Combined_post_cg_AgeGender')
plt.show()   # Display

**SVM Classifier**

In [ ]:
svclassifier = SVC(C= 1, kernel='linear', degree=3, gamma="auto")
svclassifier.fit(Train_X, Train_Y)

In [ ]:
# Make prediction
y_pred = svclassifier.predict(Test_X)
accuracy = accuracy_score(Test_Y, y_pred)
#Evaluate Model
precision, recall, fscore, support = score(Test_Y, y_pred)
print("Accuracy: ", accuracy.mean(), "Recall: ", recall.mean(), "Precision: ", 
    precision.mean(), "f1-score: ", fscore.mean())

**SVM with cross validation**

In [ ]:
svclassifier = SVC(C= 1, kernel='linear', degree=3, gamma="auto")

accuracy = performCV(Train_X, Train_Y, svclassifier, 5, 'accuracy')
recall = performCV(Train_X, Train_Y, svclassifier, 5, 'recall')
precision = performCV(Train_X, Train_Y, svclassifier, 5, 'precision')
f1score = performCV(Train_X, Train_Y, svclassifier, 5, 'f1')

In [ ]:
print("Accuracy: ", accuracy.mean(), "Recall: ", recall.mean(), "Precision: ", 
      precision.mean(), "f1-score: ", f1score.mean())

In [ ]:
accuracy = performCV(Train_X, Train_Y, svclassifier, 10, 'accuracy')
recall = performCV(Train_X, Train_Y, svclassifier, 10, 'recall')
precision = performCV(Train_X, Train_Y, svclassifier, 10, 'precision')
f1score = performCV(Train_X, Train_Y, svclassifier, 10, 'f1')

In [ ]:
print("Accuracy: ", accuracy.mean(), "Recall: ", recall.mean(), "Precision: ", 
      precision.mean(), "f1-score: ", f1score.mean())